In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


In [2]:
img_width, img_height = 150, 150

In [3]:
train_data_dir = '../Accident-Images-Analysis-Dataset/Accident Images Analysis Dataset/Accident -Detection'
validation_data_dir = './valid'

In [4]:
nb_train_samples = 4898
nb_validation_samples =2342
epochs = 50
batch_size = 16

In [5]:
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save('first_try.h5')






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 4898 images belonging to 2 classes.
Found 495 images belonging to 2 classes.



Epoch 1/50





306/306 [==============================] - 59s 192ms/step - loss: 0.5310 - acc: 0.7273 - val_loss: 3.3474 - val_acc: 0.5159
Epoch 2/50
306/306 [==============================] - 37s 119ms/step - loss: 0.4017 - acc: 0.8313 - val_loss: 3.3729 - val_acc: 0.4955
Epoch 3/50
306/306 [==============================] - 36s 119ms/step - loss: 0.3502 - acc: 0.8517 - val_loss: 2.3615 - val_acc: 0.4843
Epoch 4/50
306/306 [==============================] - 36s 117ms/step - loss: 0.3084 - acc: 0.8748 - val_loss: 2.7577 - val_acc: 0.5206
Epoch 5/50
306/306 [==============================] - 37s 120ms/step - loss: 0.2686 - acc: 0.8920 - val_loss: 2.3792 - val_acc: 0.4616
Epoch 6/50
306/306

306/306 [==============================] - 35s 115ms/step - loss: 0.1023 - acc: 0.9606 - val_loss: 5.9639 - val_acc: 0.5598
Epoch 35/50
306/306 [==============================] - 36s 119ms/step - loss: 0.0717 - acc: 0.9769 - val_loss: 5.4728 - val_acc: 0.5249
Epoch 36/50
306/306 [==============================] - 38s 124ms/step - loss: 0.0826 - acc: 0.9687 - val_loss: 4.8664 - val_acc: 0.5238
Epoch 37/50
306/306 [==============================] - 38s 123ms/step - loss: 0.0819 - acc: 0.9687 - val_loss: 5.6922 - val_acc: 0.5461
Epoch 38/50
306/306 [==============================] - 36s 119ms/step - loss: 0.0732 - acc: 0.9712 - val_loss: 5.7543 - val_acc: 0.5296
Epoch 39/50
306/306 [==============================] - 38s 123ms/step - loss: 0.0609 - acc: 0.9792 - val_loss: 5.5617 - val_acc: 0.5534
Epoch 40/50
306/306 [==============================] - 36s 117ms/step - loss: 0.0914 - acc: 0.9677 - val_loss: 5.9048 - val_acc: 0.5478
Epoch 41/50
306/306 [==============================] - 38s 1

In [24]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'fc_model_1.h5'
train_data_dir = '../Accident-Images-Analysis-Dataset/Accident Images Analysis Dataset/Accident -Detection'
validation_data_dir = './train'
nb_train_samples = 4896
nb_validation_samples = 2336
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train_4.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation_4.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train_4.npy','rb'))
    train_labels = np.array(
        [0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load(open('bottleneck_features_validation_4.npy','rb'))
    validation_labels = np.array(
        [0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Found 4898 images belonging to 2 classes.
Found 2341 images belonging to 2 classes.
Train on 4896 samples, validate on 2336 samples
Epoch 1/50
4896/4896 [==============================] - 3s 524us/step - loss: 0.4246 - acc: 0.8309 - val_loss: 4.3530 - val_acc: 0.5060
Epoch 2/50
4896/4896 [==============================] - 2s 327us/step - loss: 0.2822 - acc: 0.8911 - val_loss: 5.6267 - val_acc: 0.5141
Epoch 3/50
4896/4896 [==============================] - 2s 333us/step - loss: 0.2576 - acc: 0.9083 - val_loss: 6.7272 - val_acc: 0.5068
Epoch 4/50
4896/4896 [==============================] - 2s 316us/step - loss: 0.2445 - acc: 0.9154 - val_loss: 6.7326 - val_acc: 0.5094
Epoch 5/50
4896/4896 [==============================] - 2s 336us/step - loss: 0.2258 - acc: 0.9187 - val_loss: 7.0839 - val_acc: 0.5081
Epoch 6/50
4896/4896 [==============================] - 2s 318us/step - loss: 0.2207 - acc: 0.9234 - val_loss: 7.2492 - val_acc: 0.5068
Epoch 7/50
4896/4896 [==============================

In [7]:
from numba import cuda
cuda.select_device(0)
cuda.close()

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense

# path to the model weights files.
weights_path = 'vgg16_weights.h5'
top_model_weights_path = 'fc_model_1.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '../Accident-Images-Analysis-Dataset/Accident Images Analysis Dataset/Accident -Detection'
validation_data_dir = './train'
nb_train_samples = 24896
nb_validation_samples = 2336
epochs = 50
batch_size = 16

# build the VGG16 network
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:15]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

model.summary()

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    verbose=2)
model.save('first_net.h5')

Model loaded.
Found 4898 images belonging to 2 classes.
Found 2341 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
____________________________________________